In [1]:
# %pip install snowflake-connector-python
# %pip install pyarrow==8.0.0
# %pip install pandas
# %pip install confluent-kafka
# %pip install python-dotenv

In [2]:
import re
import warnings

import pandas as pd
import snowflake.connector

from snowflake_helpers import *
from transformation_helpers import *

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SyntaxWarning)


/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:198: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if hr is 0:


In [3]:
conn = connect_to_snowflake()
cs = conn.cursor()

Connected to Snowflake with user: admin


In [4]:
create_production_schema(cs)
use_production_schema(cs)

Schema: yusra_stories_production created.


In [5]:
df = get_logs_table_as_df(cs)

Logs table from staging schema loaded into dataframe.


In [6]:
formatted_df = get_joined_formatted_df(df)

In [7]:
user_df = get_users_df(formatted_df)

Users dataframe ready to be added to snowflake production schema.


## RIDE DF

In [8]:
staging_rides_df = get_staging_rides_df(formatted_df)

In [9]:
rides_df = get_final_rides_df(staging_rides_df)

Rides dataframe ready to be added to snowflake production schema.


In [10]:
create_users_table(cs)
write_pandas_to_users_table(conn, user_df)

Empty users table created.
Dataframe written to production schema table: USERS


In [11]:
rides_df.dtypes

ride_id                           object
user_id                            Int64
start_time                datetime64[ns]
end_time                  datetime64[ns]
total_duration                    object
max_heart_rate_bpm                 int64
min_heart_rate_bpm                 int64
avg_heart_rate_bpm                 int64
avg_resistance                     int64
avg_rpm                            int64
total_power_kilojoules           float64
dtype: object

In [12]:
create_rides_table(cs)


Empty rides table created.


In [13]:
rides_df

,ride_id,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
2,1,4612,2022-10-06 15:49:21,2022-10-06 15:58:00,0:08:37,175,70,126,42,39,13.51
1039,2,4614,2022-10-06 15:58:01,2022-10-06 16:06:42,0:08:40,139,62,91,41,52,13.10
2082,3,4614,2022-10-06 16:06:44,2022-10-06 16:15:25,0:08:40,180,83,136,41,40,14.53
3125,4,4614,2022-10-06 16:15:27,2022-10-06 16:24:07,0:08:39,134,65,100,41,45,11.08
4166,5,4614,2022-10-06 16:24:08,2022-10-06 16:32:50,0:08:40,166,79,121,41,43,8.76
5209,6,4614,2022-10-06 16:32:51,2022-10-06 16:40:55,0:08:03,165,82,133,42,42,13.56
6178,7,4614,2022-10-06 16:40:57,2022-10-06 16:49:34,0:08:36,171,61,125,41,43,10.42
7213,8,4614,2022-10-06 16:49:35,2022-10-06 16:58:17,0:08:40,144,78,104,41,43,12.93
8256,9,4616,2022-10-06 16:58:18,2022-10-06 17:06:58,0:08:39,142,82,113,40,45,12.42
9297,10,4616,2022-10-06 17:07:00,2022-10-06 17:15:41,0:08:40,165,66,115,40,42,10.27


In [14]:
rides_df.dtypes

ride_id                           object
user_id                            Int64
start_time                datetime64[ns]
end_time                  datetime64[ns]
total_duration                    object
max_heart_rate_bpm                 int64
min_heart_rate_bpm                 int64
avg_heart_rate_bpm                 int64
avg_resistance                     int64
avg_rpm                            int64
total_power_kilojoules           float64
dtype: object

In [15]:
write_pandas_to_rides_table(conn, rides_df)

Dataframe written to production schema table: RIDES
